In [90]:
!pip install plotly

  Using cached plotly-5.3.1-py2.py3-none-any.whl (23.9 MB)


In [1]:
import numpy as np
import pandas as pd

import plotly.express as px

import tensorflow as tf
import tensorflow.keras as keras

from sklearn.model_selection import train_test_split

In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [3]:
np.random.seed(2021)
tf.random.set_seed(2021)

In [4]:
num_classes=10

x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,test_size =0.2)

In [5]:
class Encoder(keras.Model):
    def __init__(self,n_layers=2,latent_features=2,hidden_featrues=64):
        super().__init__()
        self.latent_features=latent_features
        # First Layer
        self.conv_0=keras.layers.Conv2D(hidden_featrues, kernel_size=3, activation="relu",padding='same')
        self.dropout_0=keras.layers.Dropout(0.2)
        # Middle Layers
        self.convs=[keras.layers.Conv2D(hidden_featrues, kernel_size=3, activation="relu",padding='same') for _ in range(n_layers)]
        self.mxpools = [keras.layers.MaxPooling2D(pool_size=2) for _ in range(n_layers)]
        self.dropouts = [keras.layers.Dropout(0.2) for _ in range(n_layers)]
        # Integration Layer(Output Layer)
        WIDTH=2
        self.gap=keras.layers.AveragePooling2D(pool_size=WIDTH,)
        self.flatten=keras.layers.Flatten()
        self.dense=keras.layers.Dense(latent_features)
        
    def __call__(self,x):
        x=self.dropout_0(self.conv_0(x))
        for conv,mxp,drp in zip(self.convs,self.mxpools,self.dropouts):
            x=drp(mxp(conv(x)))
        x=self.gap(x)
        x=self.flatten(x)
        y=self.dense(x)
        return y
    
class ClassModel(keras.Model):
    def __init__(self,encoder,out_features=10,mode="dense"):
        super().__init__()
        self.encoder=encoder
        self.mode=mode
        
        self.out_weight=keras.layers.Dense(out_features,use_bias=False)
        
        if self.mode=="dense":
            self.out_func=keras.layers.Softmax()
#         if self.mode=="cos":
#             self.out_func=AddMarginProduct(latent_features,out_features,s=s,m=m)
#         if self.mode=="arc":
#             self.out_func=ArcMarginProduct(latent_features,out_features,s=s,m=m,easy_margin=easy_margin)
        
        
    def __call__(self,x,training=False):
        latent=self.encoder(x)
        y=self.out_func(self.out_weight(latent))
        
        return y

In [6]:
enc=Encoder(n_layers=2,latent_features=2,hidden_featrues=64)
model=ClassModel(enc,out_features=10,mode="dense")
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [7]:
## Recording Callbacks

class LogLatent(keras.callbacks.Callback):
    def __init__(self,encoder,test_x,test_y,skip_iters=100):
        self.df=self.df=pd.DataFrame()
        self.encoder=encoder
        self.plt_x=test_x
        self.plt_y=test_y
        self.itr=0
        self.skip_iters=skip_iters
    def on_batch_end(self,batch, logs={}):
        if self.itr%self.skip_iters==self.skip_iters-1:
            latent=self.encoder(self.plt_x)
            df_new=pd.DataFrame()
            df_new["f1"]=latent[:,0]
            df_new["f2"]=latent[:,1]
            df_new["arc"]=0.5
            df_new["iterations"]=np.repeat(self.itr,len(latent))

            df_new["label"]=self.plt_y
            self.df=self.df.append(df_new)
        self.itr+=1
    def normalize(self):
        # Normalize latent by maximum value
        self.mx_norm=np.linalg.norm(self.df[["f1","f2"]].values,axis=1).max()
        self.df[["f1",'f2']]=self.df[["f1",'f2']].apply(lambda x: x/self.mx_norm)

        # Project latents into arc
        df=self.df.copy()
        norm=np.linalg.norm(df[["f1","f2"]].values,axis=1,keepdims=True)
        df[["f1",'f2']]=df[["f1",'f2']].values/norm
        df["arc"]=df["arc"].map({0.5:0.9},na_action=None)
        
        self.df=self.df.append(df)
        


In [8]:
batch_size = 128
epochs = 10
plt_x,plt_y=x_test[:batch_size],y_test[:batch_size]
lg=LogLatent(enc,plt_x,plt_y.argmax(-1))

train_loader = tf.data.Dataset.from_tensor_slices((x_train,y_train)).cache().shuffle(len(x_train)//2).batch(batch_size)
val_loader = tf.data.Dataset.from_tensor_slices((x_val,y_val)).cache().shuffle(len(x_val)//2).batch(batch_size*2)


In [9]:
for ii in train_loader:
    pass
for ii in val_loader:
    pass

In [10]:
try:
    model.fit(train_loader, 
              epochs=epochs,
              validation_data=val_loader,
              callbacks=[lg])
except KeyboardInterrupt:
    print("KeyboardInterrupt")

Epoch 1/10
375/375 [==============================] - 6s 10ms/step - loss: 1.6129 - accuracy: 0.3873 - val_loss: 0.8175 - val_accuracy: 0.6994
Epoch 2/10
375/375 [==============================] - 3s 8ms/step - loss: 0.7356 - accuracy: 0.7599 - val_loss: 0.5843 - val_accuracy: 0.8304
Epoch 3/10
375/375 [==============================] - 3s 8ms/step - loss: 0.5581 - accuracy: 0.8374 - val_loss: 0.5103 - val_accuracy: 0.8560
Epoch 4/10
375/375 [==============================] - 3s 8ms/step - loss: 0.4837 - accuracy: 0.8632 - val_loss: 0.4487 - val_accuracy: 0.8737
Epoch 5/10
375/375 [==============================] - 3s 8ms/step - loss: 0.4258 - accuracy: 0.8819 - val_loss: 0.4259 - val_accuracy: 0.8846
Epoch 6/10
375/375 [==============================] - 3s 8ms/step - loss: 0.3838 - accuracy: 0.8932 - val_loss: 0.3981 - val_accuracy: 0.8907
Epoch 7/10
375/375 [==============================] - 3s 8ms/step - loss: 0.3631 - accuracy: 0.8994 - val_loss: 0.3917 - val_accuracy: 0.8907
Epoch

In [11]:
model.evaluate(x_test, y_test, batch_size=batch_size)

79/79 [==============================] - 0s 3ms/step - loss: 0.3286 - accuracy: 0.9076


[0.3286300003528595, 0.9075999855995178]

In [12]:
lg.normalize()

In [ ]:
fig=px.scatter(lg.df, x="f1", y="f2", animation_frame="iterations", color="label",opacity=lg.df.arc,
                range_x=[-1.5,1.5],
              range_y=[-1.5,1.5])
fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
)

In [15]:
fig.write_html("history/dense_tf.html")